### Great work team!👏 👏 👏

# Anostep results so far (updated daily)

Here are some quick analyses to look at the data so far :)
We start by accessing the data and cleaning it up
The top code is the simplified one.

In [1]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth

def clean_column_names(df):
    """Clean up column names by removing 'form ' prefix and replacing special characters"""
    column_mapping = {}
    
    for col in df.columns:
        new_col = col
        
        # Remove "form " prefix
        if col.lower().startswith('form '):
            new_col = col[5:]
        
        # Clean up special characters
        new_col = new_col.replace(' | ', '_').replace(': ', '_').replace(' ', '_')
        new_col = new_col.replace('__', '_').strip('_')
        
        column_mapping[col] = new_col
    
    return df.rename(columns=column_mapping), column_mapping

def main():
    # Configuration
    url = "https://www.commcarehq.org/a/atsb-project-1/api/odata/forms/v1/b81194d6348b69e2ce1880689777a556/feed"
    username = "otienobrn09@gmail.com"
    password = "Tracy@2013"
    
    print("Connecting to CommCare OData...")
    
    try:
        # Make API request
        response = requests.get(url, auth=HTTPBasicAuth(username, password), timeout=30)
        
        if response.status_code != 200:
            print(f"❌ Error {response.status_code}: {response.reason}")
            return
        
        # Process data
        data = response.json()
        
        if not data.get('value'):
            print("❌ No data found")
            return
        
        # Create DataFrame
        df = pd.DataFrame(data['value'])
        
        # Clean column names
        df, column_mapping = clean_column_names(df)
        renamed_count = len([k for k, v in column_mapping.items() if k != v])
        
        # Display results
        print(f"✅ Successfully retrieved {len(df):,} records with {len(df.columns)} columns")
        if renamed_count > 0:
            print(f"🧹 Cleaned {renamed_count} column names")
        
        # Save to CSV
        filename = "commcare_data.csv"
        df.to_csv(filename, index=False)
        print(f"💾 Data saved to {filename}")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
    except Exception as e:
        print(f"❌ Error: {e}")

if __name__ == "__main__":
    main()

Connecting to CommCare OData...
✅ Successfully retrieved 1,229 records with 59 columns
🧹 Cleaned 51 column names
💾 Data saved to commcare_data.csv


This code will now sync the data every day at 13hrs

In [ ]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
import schedule
import time
from datetime import datetime

def clean_column_names(df):
    """Clean up column names by removing 'form ' prefix and replacing special characters"""
    column_mapping = {}
    
    for col in df.columns:
        new_col = col
        
        # Remove "form " prefix
        if col.lower().startswith('form '):
            new_col = col[5:]
        
        # Clean up special characters
        new_col = new_col.replace(' | ', '_').replace(': ', '_').replace(' ', '_')
        new_col = new_col.replace('__', '_').strip('_')
        
        column_mapping[col] = new_col
    
    return df.rename(columns=column_mapping), column_mapping

def sync_data():
    """Main data sync function"""
    print(f"\n🕐 Starting sync at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Configuration
    url = "https://www.commcarehq.org/a/atsb-project-1/api/odata/forms/v1/b81194d6348b69e2ce1880689777a556/feed"
    username = "otienobrn09@gmail.com"
    password = "Tracy@2013"
    
    print("Connecting to CommCare OData...")
    
    try:
        # Make API request
        response = requests.get(url, auth=HTTPBasicAuth(username, password), timeout=30)
        
        if response.status_code != 200:
            print(f"❌ Error {response.status_code}: {response.reason}")
            return
        
        # Process data
        data = response.json()
        
        if not data.get('value'):
            print("❌ No data found")
            return
        
        # Create DataFrame
        df = pd.DataFrame(data['value'])
        
        # Clean column names
        df, column_mapping = clean_column_names(df)
        renamed_count = len([k for k, v in column_mapping.items() if k != v])
        
        # Save all data (unfiltered)
        all_data_filename = "commcare_data.csv"
        df.to_csv(all_data_filename, index=False)
        print(f"💾 All data saved to {all_data_filename}")
        
        # Filter out rows where anoph_present is '---' for cleaned dataset
        initial_rows = len(df)
        df_cleaned = df.copy()
        if 'anoph_present' in df.columns:
            df_cleaned = df_cleaned[df_cleaned['anoph_present'] != '---']
            filtered_rows = initial_rows - len(df_cleaned)
            if filtered_rows > 0:
                print(f"🗑️ Removed {filtered_rows} rows where anoph_present = '---'")
        
        # Save cleaned data
        cleaned_filename = "commcare_cleaned_data.csv"
        df_cleaned.to_csv(cleaned_filename, index=False)
        print(f"💾 Cleaned data saved to {cleaned_filename}")
        
        # Display results
        print(f"✅ Successfully processed:")
        print(f"   📊 All data: {len(df):,} records with {len(df.columns)} columns")
        print(f"   🧽 Cleaned data: {len(df_cleaned):,} records with {len(df_cleaned.columns)} columns")
        if renamed_count > 0:
            print(f"🧹 Cleaned {renamed_count} column names")
        
        print(f"✅ Sync completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
    except Exception as e:
        print(f"❌ Error: {e}")

def main():
    # Schedule daily sync at 13:15 (1 PM)
    schedule.every().day.at("13:15").do(sync_data)
    
    print("🔄 Auto-sync scheduler started!")
    print("📅 Daily sync scheduled at 13:15 (1 PM)")
    print("⚡ Running initial sync now...")
    
    # Run initial sync
    sync_data()
    
    print(f"\n⏰ Next sync scheduled for today at 13:15")
    print("🔄 Scheduler running... Press Ctrl+C to stop")
    
    # Keep the scheduler running
    while True:
        schedule.run_pending()
        time.sleep(60)  # Check every minute

if __name__ == "__main__":
    main()

🔄 Auto-sync scheduler started!
📅 Daily sync scheduled at 13:15 (1 PM)
⚡ Running initial sync now...

🕐 Starting sync at 2025-10-03 15:19:12
Connecting to CommCare OData...
💾 All data saved to commcare_data.csv
🗑️ Removed 36 rows where anoph_present = '---'
💾 Cleaned data saved to commcare_cleaned_data.csv
✅ Successfully processed:
   📊 All data: 1,220 records with 59 columns
   🧽 Cleaned data: 1,184 records with 59 columns
🧹 Cleaned 51 column names
✅ Sync completed at 2025-10-03 15:19:16

⏰ Next sync scheduled for today at 13:15
🔄 Scheduler running... Press Ctrl+C to stop


In [ ]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
from tabulate import tabulate
import os
from datetime import datetime
import json

def load_credentials():
    """Load credentials - hardcoded for now"""
    username = "otienobrn09@gmail.com"
    password = "Tracy@2013"
    
    return username, password

def fetch_commcare_data(url, username, password, timeout=30):
    """Fetch data from CommCare OData API"""
    print("Connecting to CommCare OData...")
    
    try:
        response = requests.get(
            url, 
            auth=HTTPBasicAuth(username, password),
            timeout=timeout,
            headers={'Accept': 'application/json'}
        )
        
        if response.status_code == 200:
            return response.json(), None
        else:
            error_messages = {
                401: "Authentication failed. Please check your credentials.",
                403: "Access forbidden. You may not have permission to access this data.",
                404: "Data source not found. Please check the URL.",
                429: "Too many requests. Please wait and try again.",
                500: "Server error. Please try again later."
            }
            
            error_msg = error_messages.get(
                response.status_code, 
                f"HTTP Error {response.status_code}: {response.reason}"
            )
            return None, error_msg
            
    except requests.exceptions.Timeout:
        return None, "Request timed out. The server may be slow to respond."
    except requests.exceptions.ConnectionError:
        return None, "Connection error. Please check your internet connection."
    except requests.exceptions.RequestException as e:
        return None, f"Request failed: {e}"
    except Exception as e:
        return None, f"Unexpected error: {e}"

def clean_column_names(df):
    """Remove 'form ' prefix from column names and clean them up"""
    
    # Create a mapping of old to new column names
    column_mapping = {}
    
    for col in df.columns:
        new_col = col
        
        # Remove "form " prefix (case insensitive)
        if col.lower().startswith('form '):
            new_col = col[5:]  # Remove first 5 characters ("form ")
        
        # Clean up other common patterns
        new_col = new_col.replace(' | ', '_')  # Replace " | " with "_"
        new_col = new_col.replace(': ', '_')   # Replace ": " with "_"
        new_col = new_col.replace(' ', '_')    # Replace spaces with underscores
        new_col = new_col.replace('__', '_')   # Replace double underscores with single
        new_col = new_col.strip('_')           # Remove leading/trailing underscores
        
        column_mapping[col] = new_col
    
    # Rename the columns
    df_clean = df.rename(columns=column_mapping)
    
    return df_clean, column_mapping

def analyze_dataframe(df):
    """Analyze and display basic DataFrame information"""
    print(f"\n✅ Successfully retrieved data!")
    print(f"📊 Total records: {len(df):,}")
    print(f"📋 Columns: {len(df.columns)}")
    
    return df

def display_data_info(df):
    """Display basic data information"""
    print(f"\n📊 Data Summary:")
    print("-" * 50)
    print(f"Shape: {df.shape}")

def save_data(df, base_filename="commcare_data"):
    """Save data as CSV file"""
    filename = f"{base_filename}.csv"
    
    df.to_csv(filename, index=False)
    
    size_mb = os.path.getsize(filename) / 1024 / 1024
    print(f"\n✅ Data saved to: {filename}")
    print(f"📁 File size: {size_mb:.2f} MB")

def main():
    """Main function to orchestrate the data retrieval process"""
    # Configuration
    url = "https://www.commcarehq.org/a/atsb-project-1/api/odata/forms/v1/b81194d6348b69e2ce1880689777a556/feed"
    
    # Load credentials securely
    username, password = load_credentials()
    
    # Fetch data
    data, error = fetch_commcare_data(url, username, password)
    
    if error:
        print(f"❌ {error}")
        return
    
    # Check if data contains records
    if not data or 'value' not in data or not data['value']:
        print("❌ No data found in the response")
        if data:
            print("Response structure:", list(data.keys()) if isinstance(data, dict) else type(data))
        return
    
    # Convert to DataFrame
    try:
        df = pd.DataFrame(data['value'])
    except Exception as e:
        print(f"❌ Error creating DataFrame: {e}")
        return
    
    # Clean column names
    df, column_mapping = clean_column_names(df)
    if column_mapping:
        renamed_count = len([k for k, v in column_mapping.items() if k != v])
        if renamed_count > 0:
            print(f"🧹 Cleaned {renamed_count} column names")
    
    # Analyze and display data
    df = analyze_dataframe(df)
    display_data_info(df)
    
    # Save data
    save_data(df)
    
    print(f"\n🔚 Script completed successfully!")
    print(f"📈 Final dataset: {len(df):,} rows × {len(df.columns)} columns")

if __name__ == "__main__":
    main()

In [ ]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
import json
import schedule
import time
from datetime import datetime
import logging
import threading
import sys
import os

# Configuration
url = "https://www.commcarehq.org/a/atsb-project-1/api/odata/forms/v1/b81194d6348b69e2ce1880689777a556/feed"
username = "otienobrn09@gmail.com"
password = "Tracy@2013"

# Setup logging
def setup_logging():
    """Setup logging configuration"""
    log_filename = f"commcare_sync_{datetime.now().strftime('%Y%m%d')}.log"
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler(sys.stdout)
        ]
    )
    return logging.getLogger(__name__)

logger = setup_logging()

def clean_column_names(df):
    """Remove 'form ' prefix from column names and clean them up"""
    
    # Create a mapping of old to new column names
    column_mapping = {}
    
    for col in df.columns:
        new_col = col
        
        # Remove "form " prefix (case insensitive)
        if col.lower().startswith('form '):
            new_col = col[5:]  # Remove first 5 characters ("form ")
        
        # Clean up other common patterns
        new_col = new_col.replace(' | ', '_')  # Replace " | " with "_"
        new_col = new_col.replace(': ', '_')   # Replace ": " with "_"
        new_col = new_col.replace(' ', '_')    # Replace spaces with underscores
        new_col = new_col.replace('__', '_')   # Replace double underscores with single
        new_col = new_col.strip('_')           # Remove leading/trailing underscores
        
        column_mapping[col] = new_col
    
    # Rename the columns
    df_clean = df.rename(columns=column_mapping)
    
    return df_clean, column_mapping

# def fetch_all_data_auto(max_records=10000, batch_size=100):
#     """Automatically fetch all available data with deduplication"""
#     auth = HTTPBasicAuth(username, password)
#     all_data = []
#     seen_ids = set()  # Track unique identifiers to prevent duplicates
#     offset = 0
#     consecutive_empty_batches = 0
#     max_empty_batches = 3  # Stop after 3 consecutive empty batches
    
#     logger.info(f"Starting automatic data fetch - max {max_records} records")
    
#     while len(all_data) < max_records and consecutive_empty_batches < max_empty_batches:
#         try:
#             params = {
#                 '$top': batch_size,
#                 '$skip': offset
#             }
            
#             logger.info(f"Fetching batch: records {offset} to {offset + batch_size}")
            
#             response = requests.get(url, auth=auth, params=params, timeout=60)
            
#             if response.status_code == 401:
#                 logger.error("Authentication failed - check credentials")
#                 break
#             elif response.status_code == 403:
#                 logger.error("Access forbidden - check permissions")
#                 break
#             elif response.status_code != 200:
#                 logger.error(f"HTTP {response.status_code} at offset {offset}")
#                 break
            
#             data = response.json()
            
#             if 'value' not in data:
#                 logger.error("No 'value' key in response")
#                 break
                
#             batch_data = data['value']
            
#             if not batch_data:
#                 consecutive_empty_batches += 1
#                 logger.info(f"Empty batch {consecutive_empty_batches}/{max_empty_batches} - checking for more data")
#                 offset += batch_size
#                 continue
#             else:
#                 consecutive_empty_batches = 0  # Reset counter when we get data
            
#             # Deduplicate based on a unique identifier
#             new_records = []
#             duplicates_found = 0
            
#             for record in batch_data:
#                 # Try to find a unique identifier - adjust these field names based on your data structure
#                 unique_id = None
                
#                 # Common unique identifier fields in CommCare - try these in order
#                 for id_field in ['id', '_id', 'caseid', 'form_id', 'instanceID', '__id__']:
#                     if id_field in record and record[id_field]:
#                         unique_id = str(record[id_field])
#                         break
                
#                 # If no standard ID field found, create a hash of the entire record
#                 if unique_id is None:
#                     # Sort the record items to ensure consistent hashing
#                     record_str = json.dumps(record, sort_keys=True)
#                     unique_id = str(hash(record_str))
#                     logger.warning(f"No standard ID field found, using hash: {unique_id}")
                
#                 if unique_id not in seen_ids:
#                     seen_ids.add(unique_id)
#                     new_records.append(record)
#                 else:
#                     duplicates_found += 1
            
#             if duplicates_found > 0:
#                 logger.warning(f"Found {duplicates_found} duplicate records in this batch")
            
#             all_data.extend(new_records)
#             logger.info(f"Batch complete. New unique records: {len(new_records)}, Total records: {len(all_data)}")
            
#             # Check if we got fewer records than requested (potential end of data)
#             if len(batch_data) < batch_size:
#                 logger.info("Got fewer records than requested - might be reaching end of data")
#                 # Still continue to check a few more batches in case there are gaps
            
#             offset += batch_size
            
#             # Add a small delay to be respectful to the API
#             time.sleep(0.1)
            
#         except requests.exceptions.Timeout:
#             logger.error(f"Request timeout at offset {offset}")
#             break
#         except requests.exceptions.ConnectionError:
#             logger.error(f"Connection error at offset {offset}")
#             break
#         except Exception as e:
#             logger.error(f"Error fetching batch at offset {offset}: {e}")
#             import traceback
#             logger.error(traceback.format_exc())
#             break
    
#     logger.info(f"Data fetch completed. Total unique records: {len(all_data)}")
    
#     if all_data:
#         df = pd.DataFrame(all_data)
        
#         # Additional pandas-level deduplication as a safety measure
#         initial_count = len(df)
        
#         # Try to find the best column for deduplication
#         id_columns = ['id', '_id', 'caseid', 'form_id', 'instanceID', '__id__']
#         dedup_column = None
        
#         for col in id_columns:
#             if col in df.columns:
#                 dedup_column = col
#                 break
        
#         if dedup_column:
#             df = df.drop_duplicates(subset=[dedup_column])
#             final_count = len(df)
#             if initial_count != final_count:
#                 logger.warning(f"Removed {initial_count - final_count} additional duplicates using column '{dedup_column}'")
#         else:
#             # If no ID column found, remove duplicates based on all columns
#             df = df.drop_duplicates()
#             final_count = len(df)
#             if initial_count != final_count:
#                 logger.warning(f"Removed {initial_count - final_count} additional duplicates using all columns")
        
#         return df
#     else:
#         return None

def save_daily_data(df_clean, column_mapping):
    """Save daily data with timestamp"""
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save cleaned data with timestamp
    data_filename = f"commcare_data_{timestamp}.csv"
    df_clean.to_csv(data_filename, index=False)
    logger.info(f"Daily data saved to '{data_filename}'")
    
    # Always maintain current files for other scripts
    current_data_filename = "commcare_cleaned_data.csv"
    df_clean.to_csv(current_data_filename, index=False)
    logger.info(f"Current data updated: '{current_data_filename}'")
    
    # Save column mapping
    mapping_df = pd.DataFrame(list(column_mapping.items()), 
                             columns=['Original_Column', 'Cleaned_Column'])
    mapping_filename = f"commcare_column_mapping_{timestamp}.csv"
    mapping_df.to_csv(mapping_filename, index=False)
    
    current_mapping_filename = "commcare_column_mapping.csv"
    mapping_df.to_csv(current_mapping_filename, index=False)
    
    # Save metadata
    metadata = {
        'sync_timestamp': timestamp,
        'sync_datetime': datetime.now().isoformat(),
        'total_records': len(df_clean),
        'total_columns': len(df_clean.columns),
        'columns': list(df_clean.columns),
        'data_types': {col: str(dtype) for col, dtype in df_clean.dtypes.items()},
        'null_counts': {col: int(df_clean[col].isnull().sum()) for col in df_clean.columns}
    }
    
    metadata_filename = f"commcare_metadata_{timestamp}.json"
    with open(metadata_filename, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    current_metadata_filename = "commcare_metadata.json"
    with open(current_metadata_filename, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    logger.info(f"Metadata saved to '{metadata_filename}'")
    
    return {
        'data_file': data_filename,
        'current_data_file': current_data_filename,
        'mapping_file': mapping_filename,
        'metadata_file': metadata_filename,
        'records_count': len(df_clean)
    }

# def daily_sync_job():
#     """Daily synchronization job"""
#     logger.info("=" * 60)
#     logger.info("DAILY COMMCARE DATA SYNC STARTED")
#     logger.info("=" * 60)
    
#     try:
#         # Test connection first
#         auth = HTTPBasicAuth(username, password)
#         test_response = requests.get(url, auth=auth, timeout=30, params={'$top': 1})
        
#         if test_response.status_code != 200:
#             logger.error(f"Connection test failed: HTTP {test_response.status_code}")
#             return
        
#         logger.info("Connection test successful")
        
#         # Fetch all data
#         df = fetch_all_data_auto()
        
#         if df is None:
#             logger.error("Failed to fetch data from CommCare")
#             return
        
#         logger.info(f"Successfully fetched {len(df)} records with {len(df.columns)} columns")
        
#         # Clean column names
#         logger.info("Cleaning column names...")
#         df_clean, column_mapping = clean_column_names(df)
        
#         # Save data
#         logger.info("Saving data...")
#         files = save_daily_data(df_clean, column_mapping)
        
#         logger.info("=" * 60)
#         logger.info("DAILY SYNC COMPLETED SUCCESSFULLY")
#         logger.info(f"Records processed: {files['records_count']}")
#         logger.info(f"Files updated:")
#         for desc, filename in files.items():
#             if desc.endswith('_file'):
#                 logger.info(f"  • {filename}")
#         logger.info("=" * 60)
        
#     except Exception as e:
#         logger.error(f"Daily sync failed: {e}")
#         import traceback
#         logger.error("Full error details:")
#         logger.error(traceback.format_exc())

def show_status():
    """Show current sync status"""
    print("\n📊 COMMCARE DATA SYNC STATUS")
    print("=" * 50)
    
    # Check if current files exist
    current_files = {
        'Data': 'commcare_cleaned_data.csv',
        'Mapping': 'commcare_column_mapping.csv', 
        'Metadata': 'commcare_metadata.json'
    }
    
    for file_type, filename in current_files.items():
        if os.path.exists(filename):
            mod_time = datetime.fromtimestamp(os.path.getmtime(filename))
            size = os.path.getsize(filename)
            print(f"✅ {file_type:8}: {filename}")
            print(f"   Last updated: {mod_time.strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"   Size: {size:,} bytes")
        else:
            print(f"❌ {file_type:8}: {filename} (not found)")
    
    # Check recent log files
    log_files = [f for f in os.listdir('.') if f.startswith('commcare_sync_') and f.endswith('.log')]
    if log_files:
        latest_log = max(log_files, key=lambda f: os.path.getmtime(f))
        mod_time = datetime.fromtimestamp(os.path.getmtime(latest_log))
        print(f"\n📋 Latest log: {latest_log}")
        print(f"   Last modified: {mod_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Show next scheduled run
    jobs = schedule.get_jobs()
    if jobs:
        next_run = min(job.next_run for job in jobs)
        print(f"\n⏰ Next scheduled sync: {next_run.strftime('%Y-%m-%d %H:%M:%S')}")
    else:
        print(f"\n⏰ No scheduled syncs configured")

def main():
    print("🔄 CommCare Auto-Sync Data Loader")
    print("=" * 60)
    print("Automatic daily sync scheduled for 9:15 AM")
    print("Press Ctrl+C to stop the scheduler")
    print("=" * 60)
    
    # Schedule daily job at 9:15 AM
    schedule.every().day.at("09:15").do(daily_sync_job)
    
    # Show initial status
    show_status()
    
    try:
        logger.info("Starting CommCare data scheduler...")
        logger.info("Daily sync scheduled for 9:15 AM")
        logger.info("Press Ctrl+C to stop")
        
        # Run scheduler loop
        while True:
            schedule.run_pending()
            time.sleep(60)  # Check every minute
            
    except KeyboardInterrupt:
        print("\n⏹️  Scheduler stopped by user")
        logger.info("Scheduler stopped by user")
        print("👋 Goodbye!")

if __name__ == "__main__":
    main()

In [ ]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
from datetime import datetime
import logging

# Configuration
url = "https://www.commcarehq.org/a/atsb-project-1/api/odata/forms/v1/b81194d6348b69e2ce1880689777a556/feed"
username = "otienobrn09@gmail.com"
password = "Tracy@2013"

# Simple logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

def fetch_and_process_data():
    """Fetch data from CommCare API, flatten nested structures, and deduplicate by formid"""
    
    auth = HTTPBasicAuth(username, password)
    all_data = []
    offset = 0
    batch_size = 100
    
    logger.info("Starting data fetch from CommCare API...")
    
    # # Fetch all data in batches
    # while True:
    #     try:
    #         params = {'$top': batch_size, '$skip': offset}
    #         response = requests.get(url, auth=auth, params=params, timeout=60)
            
    #         if response.status_code != 200:
    #             logger.error(f"HTTP {response.status_code} - stopping fetch")
    #             break
            
    #         data = response.json()
    #         batch_data = data.get('value', [])
            
    #         if not batch_data:
    #             logger.info("No more data - fetch complete")
    #             break
            
    #         all_data.extend(batch_data)
    #         logger.info(f"Fetched {len(all_data)} records so far...")
            
    #         if len(batch_data) < batch_size:
    #             break
                
    #         offset += batch_size
            
    #     except Exception as e:
    #         logger.error(f"Error fetching data: {e}")
    #         break
    
    # if not all_data:
    #     logger.error("No data fetched")
    #     return None
    
    # logger.info(f"Total raw records: {len(all_data)}")
    
    # Flatten nested JSON structures
    logger.info("Flattening nested data...")
    df = pd.json_normalize(all_data)
    
    logger.info(f"After flattening: {len(df)} rows, {len(df.columns)} columns")
    
    # Find formid column for deduplication
    formid_columns = [col for col in df.columns if 'formid' in col.lower() or 'form_id' in col.lower() or 'instanceid' in col.lower()]
    
    if formid_columns:
        formid_col = formid_columns[0]
        logger.info(f"Using '{formid_col}' for deduplication")
        
        # Remove duplicates
        original_count = len(df)
        df = df.drop_duplicates(subset=[formid_col], keep='first')
        final_count = len(df)
        
        logger.info(f"Deduplicated: {original_count} -> {final_count} records")
    else:
        logger.warning("No formid column found - skipping deduplication")
    
    return df

def clean_column_names(df):
    """Clean up column names"""
    df = df.copy()
    
    # Rename columns
    new_columns = {}
    for col in df.columns:
        new_col = col
        if col.lower().startswith('form '):
            new_col = col[5:]  # Remove "form " prefix
        new_col = new_col.replace(' | ', '_').replace(': ', '_').replace(' ', '_')
        new_col = new_col.replace('__', '_').strip('_')
        new_columns[col] = new_col
    
    df = df.rename(columns=new_columns)
    return df

def save_data(df):
    """Save the processed data"""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save main data file
    main_filename = "commcare_cleaned_data.csv"
    timestamped_filename = f"commcare_data_{timestamp}.csv"
    
    df.to_csv(main_filename, index=False)
    df.to_csv(timestamped_filename, index=False)
    
    logger.info(f"Data saved to: {main_filename}")
    logger.info(f"Backup saved to: {timestamped_filename}")
    
    return main_filename

def main():
    """Main execution function"""
    print("🔄 CommCare Data Fetcher (Simplified)")
    print("=" * 50)
    print("Fetching, flattening, and deduplicating CommCare data...")
    
    try:
        # Fetch and process data
        df = fetch_and_process_data()
        
        if df is None:
            print("❌ Failed to fetch data")
            return
        
        # Clean column names
        df = clean_column_names(df)
        
        # Save data
        filename = save_data(df)
        
        # Show results
        print(f"\n✅ SUCCESS!")
        print(f"📊 Records: {len(df)}")
        print(f"📋 Columns: {len(df.columns)}")
        print(f"📁 Saved to: {filename}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        logger.error(f"Main execution error: {e}")

if __name__ == "__main__":
    main()